In [12]:
#Overview

import tensorflow as tf

x = tf.constant([[1., 2., 3.],
                 [4., 5., 6.]])

x.shape, x.dtype

(TensorShape([2, 3]), tf.float32)

In [2]:
x @ tf.transpose(x)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [3]:
tf.concat([x,x,x],axis=0)

<tf.Tensor: shape=(6, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.],
       [1., 2., 3.],
       [4., 5., 6.],
       [1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [13]:
tf.nn.softmax(x,axis=-1)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.09003057, 0.24472848, 0.66524094],
       [0.09003057, 0.24472848, 0.66524094]], dtype=float32)>

In [11]:
tf.argmax(x,axis=-1)

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([2, 2], dtype=int64)>

In [9]:
from tensorflow import keras
keras.backend.argmax(x,axis=-1)

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([2, 2], dtype=int64)>

In [14]:
tf.reduce_sum(x)

<tf.Tensor: shape=(), dtype=float32, numpy=21.0>

In [15]:
# tf.convert_to_tensor()

In [18]:
class SimpleModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.a_variable = tf.Variable(5.0, name="train_me")
    self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
  def __call__(self, x):
    return self.a_variable * x + self.non_trainable_variable

simple_module = SimpleModule(name="simple")
simple_module(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

In [ ]:
#tf.checkpoint.CheckpointManager

In [19]:
#sequential model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


model = keras.Sequential(
[
    layers.Dense(2,activation='relu',name='layer1'),
    layers.Dense(3,activation='relu',name='layer2'),
    layers.Dense(4,name='layer3')
])

x = tf.ones((3,3))
y = model(x)


# layer1 = layers.Dense(2, activation='relu',name='layer1')
# layer2 = layers.Dense(3, activation='relu',name='layer2')
# layer3 = layers.Dense(4, activation='relu',name='layer3')

# x = tf.ones((3,3))
# y = layer3(layer2(layer1(x)))

In [20]:
model.layers

In [21]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2,activation='relu'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.layers

In [23]:
model = keras.Sequential()
model.add(layers.Dense(2,activation='relu',input_shape=(4,)))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [24]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

In [25]:
features

[<tf.Tensor: shape=(1, 123, 123, 32), dtype=float32, numpy=
 array([[[[0.        , 0.6472995 , 0.        , ..., 0.        ,
           0.        , 0.00965215],
          [0.        , 0.6472995 , 0.        , ..., 0.        ,
           0.        , 0.00965215],
          [0.        , 0.6472995 , 0.        , ..., 0.        ,
           0.        , 0.00965215],
          ...,
          [0.        , 0.6472995 , 0.        , ..., 0.        ,
           0.        , 0.00965215],
          [0.        , 0.6472995 , 0.        , ..., 0.        ,
           0.        , 0.00965215],
          [0.        , 0.6472995 , 0.        , ..., 0.        ,
           0.        , 0.00965215]],
 
         [[0.        , 0.6472995 , 0.        , ..., 0.        ,
           0.        , 0.00965215],
          [0.        , 0.6472995 , 0.        , ..., 0.        ,
           0.        , 0.00965215],
          [0.        , 0.6472995 , 0.        , ..., 0.        ,
           0.        , 0.00965215],
          ...,
       

In [26]:
model = keras.Sequential([
    keras.Input(shape=(784)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])

# Presumably you would want to first load pre-trained weights.
model.load_weights(...)

# Freeze all layers except the last one.
for layer in model.layers[:-1]:
  layer.trainable = False

# Recompile and train (this will only update the weights of the last layer).
model.compile(...)
model.fit(...)

AttributeError: 'ellipsis' object has no attribute 'endswith'

In [27]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model.fit(x_train, y_train, batch_size=64, epochs=2, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/2
750/750 [==============================] - 5s 6ms/step - loss: 0.7648 - accuracy: 0.7703 - val_loss: 0.2685 - val_accuracy: 0.9203
Epoch 2/2
750/750 [==============================] - 1s 2ms/step - loss: 0.2335 - accuracy: 0.9327 - val_loss: 0.1917 - val_accuracy: 0.9448
313/313 - 0s - loss: 0.1895 - accuracy: 0.9430
Test loss: 0.18948808312416077
Test accuracy: 0.9430000185966492
